In [2]:
'''
Process anomaly detection using word2vec modelling and one class classification algorithms
'''

import os
import time
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
from sklearn import metrics
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter
from sys import warnoptions


E:\anaconda3\envs\nlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
if not warnoptions:
    simplefilter("ignore", ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

In [4]:
n_workers = cpu_count()

# define functions

In [6]:
def read_log(path, log):
    '''
    Reads event log and preprocess it
    only need three collumns: case_id, activity_name, label
    '''
    df_raw = pd.read_csv(f'{path}/{log}')
    df_raw['event_processed'] = df_raw['activity_name'].str.replace(' ', '-')
    labels = [1 if x == 'normal' else -1 for x in df_raw['label']]
    df_raw['label'] = labels
    df_proc = df_raw[['case_id', 'event_processed', 'label']]
    del df_raw
    return df_proc

def cases_y_list(df):
    '''
    Creates a list of cases for model training
    '''
    cases, y = [], []
    for group in df.groupby('case_id'):
        events = list(group[1].event_processed)
        cases.append([''.join(x) for x in events])
        y.append(list(group[1].label)[0])

    return cases, y

def create_models(cases, size, window, min_count):
    '''
    Creates a word2vec model
    '''
    model = Word2Vec(vector_size=size, window=window, min_count=min_count, workers=n_workers)
    model.build_vocab(cases)
    model.train(cases, total_examples=len(cases), epochs=10)

    return model

def average_feature_vector(cases, model):
    '''
    Computes average feature vector for each trace
    '''
    vectors = []
    for case in cases:
        case_vector = []
        for token in case:
            try:
                case_vector.append(model.wv[token])
            except KeyError:
                pass
        vectors.append(np.array(case_vector).mean(axis=0))

    return vectors

def compute_metrics(y_true, y_pred):
    '''
    Computes performance metrics
    '''
    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)

    return acc, f1, precision, recall



# algorithm test

## preprocess dataset

In [9]:
scl = StandardScaler()
path = 'algorithm test'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    start_time_log = time.time()

    # reads event log
    df = read_log(path, log)
    print(df[:10])
    # process cases and labels
    cases, y = cases_y_list(df)
    del df
    

       case_id       event_processed  label
0  1-147898401                     ▶     -1
1  1-147898401  Accepted+In-Progress     -1
2  1-147898401         Accepted+Wait     -1
3  1-147898401  Accepted+In-Progress     -1
4  1-147898401  Accepted+In-Progress     -1
5  1-147898401                     ■     -1
6  1-165554831                     ▶      1
7  1-165554831  Accepted+In-Progress      1
8  1-165554831         Accepted+Wait      1
9  1-165554831  Accepted+In-Progress      1


In [11]:
type(cases), type(y), cases[:2], y[:2]

(list,
 list,
 [['▶',
   'Accepted+In-Progress',
   'Accepted+Wait',
   'Accepted+In-Progress',
   'Accepted+In-Progress',
   '■'],
  ['▶', 'Accepted+In-Progress', 'Accepted+Wait', 'Accepted+In-Progress', '■']],
 [-1, 1])

## word2vec

In [12]:
size_param = 50
window_param = 1

# generate model
model = create_models(cases, size_param, window_param, 1)

# calculating the average feature vector for each sentence (trace)
vectors = average_feature_vector(cases, model)
# normalization
vectors = scl.fit_transform(vectors)

X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1) # train:test = 7:3
X_train = np.array(X_train)
positive_mask = np.array(y_train) == 1

In [21]:
type(positive_mask), positive_mask[:20]

(numpy.ndarray,
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False, False,  True,  True,  True,  True,
        False,  True]))

In [28]:
print(y_train, len(y_train))
print('--------------------------------------')
print(y_test, len(y_test))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1,

In [30]:
X_train[0], X_test[0]

(array([-1.22692623,  1.00428581,  0.28060523,  1.07735325,  0.22744788,
        -1.17224543,  0.91964797,  0.73036757, -0.79020284, -0.63650715,
         0.79020372, -1.05438557, -1.31613593,  0.89587838, -0.41843719,
         1.12872638,  0.75032342,  0.91804593, -0.80485012, -0.79815627,
         1.1319112 ,  1.1740925 ,  0.89881298, -1.12044899,  0.97431714,
         0.02780866,  0.49235555,  0.8090252 , -0.83741003, -0.99357185,
        -0.1436161 , -0.6173456 ,  1.14165165, -0.72059164, -0.53689972,
        -0.11164452,  1.00143049, -0.64942156,  0.85941117,  1.2827898 ,
        -0.379233  ,  0.11061149, -1.11500988, -0.79305655,  0.82046941,
         0.93531167, -1.54337411,  0.37490259,  1.05134765,  1.05536402]),
 array([ 0.83366958, -0.65793188,  0.88332688, -0.7839868 ,  0.49706839,
         0.95772554, -0.74202515, -0.74776155,  1.23141793,  0.183206  ,
        -0.5784201 ,  1.41048809,  0.29301757, -1.49443766,  0.88030033,
        -0.87589886, -0.86909897, -1.26914345,  0

## LOF

### train

In [31]:
# LOF
LOF_neighbors_param = 1
LOF_contamination_param = 0.01

lof = LocalOutlierFactor(n_neighbors=LOF_neighbors_param, contamination=LOF_contamination_param, novelty=True, n_jobs=n_workers)
lof.fit(X_train[positive_mask])

LocalOutlierFactor(contamination=0.01, n_jobs=12, n_neighbors=1, novelty=True)

### predict

In [32]:
y_pred = lof.predict(X_test)

In [33]:
y_pred

array([ 1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

### compute predict metrics

In [35]:
# 预测的异常情况，实际的异常情况
acc, fscore, precision, recall = compute_metrics(y_test, y_pred)

print(log_name, 'LOF', ' fscore=', fscore, ' precision=', precision, ' recall=', recall, ' acc=', acc)


bpic13-0.3-3 LOF  fscore= 0.8811188811188811  precision= 0.7974683544303798  recall= 0.984375  acc= 0.7926829268292683


### 问题：各个算法的训练数据和测试数据是否一样？

# specified params

In [60]:
scl = StandardScaler()
# path = 'data_test'
# path = 'second_test_error'
path = 'preprocessed_csv_BPIC13OpenProblems'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    start_time_log = time.time()
    print('log_name=', log_name)
    # reads event log
    df = read_log(path, log)

    # process cases and labels
    cases, y = cases_y_list(df)
    del df

#     在BPM应用程序中，建议使用更小的向量50和窗口大小
#     for size_param in [50, 100, 150, 200, 250, 300, 400, 500, 750, 1000]: # word2vec的向量大小
#         for window_param in [1, 3, 5, 10]: # word2vec的窗口大小，论文实验表明窗口大小不影响性能。

    # 使用论文中通过实验获取的最佳向量大小和窗口大小
    size_param = 50
    window_param = 1
    
    # generate model
    model = create_models(cases, size_param, window_param, 1)

    # calculating the average feature vector for each sentence (trace)
    vectors = average_feature_vector(cases, model)
    # normalization
    vectors = scl.fit_transform(vectors)

    X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1)
    X_train = np.array(X_train)
    positive_mask = np.array(y_train) == 1
    
#     LOF_neighbors_param = 1
#     LOF_contamination_param = 0.01
#     start_time = time.time()
#     lof = LocalOutlierFactor(n_neighbors=LOF_neighbors_param, contamination=LOF_contamination_param, novelty=True, n_jobs=n_workers)
#     lof.fit(X_train[positive_mask])
#     y_pred = lof.predict(X_test)
#     acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
#     elapsed_time = time.time() - start_time
#     print(log_name, 'LOF', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
#     print('---------------------------------------------')
    
    SVM_c_param = 1000
    SVM_kernel_param = 'poly'
    SVM_gamma_param = 'scale'
    n_estimators = 10
    start_time = time.time()
    svm = BaggingClassifier(base_estimator=SVC(kernel=SVM_kernel_param, C=SVM_c_param, gamma=SVM_gamma_param, max_iter=1000000), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=n_workers)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    elapsed_time = time.time() - start_time
    print(log_name, 'SVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('---------------------------------------------')
             
#     OCSVM_nu_param = 0.01
#     OCSVM_kernel_param = 'rbf'
#     OCSVM_gamma_param = 'scale'
#     start_time = time.time()
#     ocsvm = OneClassSVM(nu=OCSVM_nu_param, kernel=OCSVM_kernel_param, gamma=OCSVM_gamma_param, max_iter=1000000)
#     ocsvm.fit(X_train[positive_mask])
#     y_pred = ocsvm.predict(X_test)
#     acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
#     elapsed_time = time.time() - start_time
#     print(log_name, 'OCSVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)

#     elapsed_time_log = time.time() - start_time_log
#     print('Finished log', log_name, 'with', elapsed_time_log, 'seconds')
    # print()


log_name= BPIC13ClosedProblems-0.1
BPIC13ClosedProblems-0.1 SVM  acc= 0.9105145413870246  fscore= 0.9527186761229314  precision= 0.9159090909090909  recall= 0.9926108374384236
---------------------------------------------
log_name= BPIC13OpenProblems-0.1


ValueError: Invalid input - all samples with positive weights have the same label.

# test preprocessed

In [8]:
scl = StandardScaler()
path = 'preprocessed_csv_for_test'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    start_time_log = time.time()

    # reads event log
    df = read_log(path, log)

    # process cases and labels
    cases, y = cases_y_list(df)
    del df

#     在BPM应用程序中，建议使用更小的向量50和窗口大小
#     for size_param in [50, 100, 150, 200, 250, 300, 400, 500, 750, 1000]: # word2vec的向量大小
#         for window_param in [1, 3, 5, 10]: # word2vec的窗口大小，论文实验表明窗口大小不影响性能。

    # 使用论文中通过实验获取的最佳向量大小和窗口大小
    size_param = 50
    window_param = 1
    
    # generate model
    model = create_models(cases, size_param, window_param, 1)

    # calculating the average feature vector for each sentence (trace)
    vectors = average_feature_vector(cases, model)
    # normalization
    vectors = scl.fit_transform(vectors)

    X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1)
    X_train = np.array(X_train)
    positive_mask = np.array(y_train) == 1

#     for neighbors_param in [1, 10, 25, 50, 100, 250]: # 论文已实验说明neighbors个数越少越好，为1
#         for contamination_param in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 'auto']: # 越高越差
    LOF_neighbors_param = 1
    LOF_contamination_param = 0.01
    start_time = time.time()
    lof = LocalOutlierFactor(n_neighbors=LOF_neighbors_param, contamination=LOF_contamination_param, novelty=True, n_jobs=n_workers)
    lof.fit(X_train[positive_mask])
    y_pred = lof.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    elapsed_time = time.time() - start_time
    print(log_name, 'LOF', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('---------------------------------------------')
    
    SVM_c_param = 1000
    SVM_kernel_param = 'poly'
    SVM_gamma_param = 'scale'
#     for c_param in [0.1, 1, 10, 100, 1000, 10000, 100000]: # 论文中最佳参数为1000
#         for kernel_param in ['poly', 'rbf', 'sigmoid']: # 论文中最佳参数为poly
#             for gamma_param in ['scale', 'auto']: # 论文中最佳参数为scale
    n_estimators = 10
    start_time = time.time()
    svm = BaggingClassifier(base_estimator=SVC(kernel=SVM_kernel_param, C=SVM_c_param, gamma=SVM_gamma_param, max_iter=1000000), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=n_workers)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    elapsed_time = time.time() - start_time
    print(log_name, 'SVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('---------------------------------------------')
             
    OCSVM_nu_param = 0.01
#     for nu_param in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:# 论文中最佳参数为0.01
    for kernel_param in ['poly', 'rbf', 'sigmoid']: # 'rbf', 'sigmoid'
        for gamma_param in ['scale', 'auto']: # auto
            start_time = time.time()
            ocsvm = OneClassSVM(nu=OCSVM_nu_param, kernel=kernel_param, gamma=gamma_param, max_iter=1000000)
            ocsvm.fit(X_train[positive_mask])
            y_pred = ocsvm.predict(X_test)
            acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
            elapsed_time = time.time() - start_time
#             print(log_name, 'OCSVM', size_param, window_param, OCSVM_nu_param, kernel_param, gamma_param, acc, fscore, precision, recall, elapsed_time)
            print(log_name, 'OCSVM', kernel_param, gamma_param, 
                  ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('----------------------------------------------------------------')
    elapsed_time_log = time.time() - start_time_log
    print('Finished log', log_name, 'with', elapsed_time_log, 'seconds')
    # print()


BPIC20PrepaidTravelCost-0.05 LOF  acc= 0.9603174603174603  fscore= 0.9790444258172675  precision= 0.9717138103161398  recall= 0.9864864864864865
---------------------------------------------
BPIC20PrepaidTravelCost-0.05 SVM  acc= 0.9380952380952381  fscore= 0.9679539852095317  precision= 0.9424  recall= 0.9949324324324325
---------------------------------------------
BPIC20PrepaidTravelCost-0.05 OCSVM poly scale  acc= 0.36984126984126986  fscore= 0.5116851168511686  precision= 0.9411764705882353  recall= 0.35135135135135137
BPIC20PrepaidTravelCost-0.05 OCSVM poly auto  acc= 0.373015873015873  fscore= 0.5153374233128833  precision= 0.9417040358744395  recall= 0.3547297297297297
BPIC20PrepaidTravelCost-0.05 OCSVM rbf scale  acc= 0.8063492063492064  fscore= 0.8870370370370371  precision= 0.9815573770491803  recall= 0.8091216216216216
BPIC20PrepaidTravelCost-0.05 OCSVM rbf auto  acc= 0.8920634920634921  fscore= 0.9401408450704225  precision= 0.9816176470588235  recall= 0.902027027027027
BP

# iterator params

In [5]:
scl = StandardScaler()
path = 'data'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    start_time_log = time.time()

    # reads event log
    df = read_log(path, log)

    # process cases and labels
    cases, y = cases_y_list(df)
    del df

#     在BPM应用程序中，建议使用更小的向量50和窗口大小
    for size_param in [50, 100, 150, 200, 250, 300, 400, 500, 750, 1000]: # word2vec的向量大小
        for window_param in [1, 3, 5, 10]: # word2vec的窗口大小，论文实验表明窗口大小不影响性能。
            # generate model
            model = create_models(cases, size_param, window_param, 1)

            # calculating the average feature vector for each sentence (trace)
            vectors = average_feature_vector(cases, model)
            # normalization
            vectors = scl.fit_transform(vectors)

            X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1)
            X_train = np.array(X_train)
            positive_mask = np.array(y_train) == 1

            for neighbors_param in [1, 10, 25, 50, 100, 250]: # 论文已实验说明neighbors个数越少越好，为1
                for contamination_param in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 'auto']: # 越高越差
                    start_time = time.time()
                    lof = LocalOutlierFactor(n_neighbors=neighbors_param, contamination=contamination_param, novelty=True, n_jobs=n_workers)
                    lof.fit(X_train[positive_mask])
                    y_pred = lof.predict(X_test)
                    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
                    elapsed_time = time.time() - start_time
                    print(log_name, 'LOF', size_param, window_param, neighbors_param, contamination_param, acc, fscore, precision, recall, elapsed_time)

            for c_param in [0.1, 1, 10, 100, 1000, 10000, 100000]: # 论文中最佳参数为1000
                for kernel_param in ['poly', 'rbf', 'sigmoid']: # 论文中最佳参数为poly
                    for gamma_param in ['scale', 'auto']: # 论文中最佳参数为scale
                        n_estimators = 10
                        start_time = time.time()
                        svm = BaggingClassifier(base_estimator=SVC(kernel=kernel_param, C=c_param, gamma=gamma_param, max_iter=1000000), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=n_workers)
                        svm.fit(X_train, y_train)
                        y_pred = svm.predict(X_test)
                        acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
                        elapsed_time = time.time() - start_time
                        print(log_name, 'SVM', size_param, window_param, c_param, kernel_param, gamma_param, acc, fscore, precision, recall, elapsed_time)

            for nu_param in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:# 论文中最佳参数为0.01
                for kernel_param in ['poly', 'rbf', 'sigmoid']:
                    for gamma_param in ['scale', 'auto']:
                        start_time = time.time()
                        ocsvm = OneClassSVM(nu=nu_param, kernel=kernel_param, gamma=gamma_param, max_iter=1000000)
                        ocsvm.fit(X_train[positive_mask])
                        y_pred = ocsvm.predict(X_test)
                        acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
                        elapsed_time = time.time() - start_time
                        print(log_name, 'OCSVM', size_param, window_param, nu_param, kernel_param, gamma_param, acc, fscore, precision, recall, elapsed_time)

    elapsed_time_log = time.time() - start_time_log
    print('Finished log', log_name, 'with', elapsed_time_log, 'seconds')
    # print()


bpic13-0.3-3 LOF 50 1 1 0.01 0.7926829268292683 0.8811188811188811 0.7974683544303798 0.984375 0.23552966117858887
bpic13-0.3-3 LOF 50 1 1 0.05 0.7967479674796748 0.8798076923076924 0.8169642857142857 0.953125 0.23310613632202148
bpic13-0.3-3 LOF 50 1 1 0.1 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.2258310317993164
bpic13-0.3-3 LOF 50 1 1 0.15 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.2281177043914795
bpic13-0.3-3 LOF 50 1 1 0.2 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.23105096817016602
bpic13-0.3-3 LOF 50 1 1 0.25 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.24551081657409668
bpic13-0.3-3 LOF 50 1 1 0.3 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.22369766235351562
bpic13-0.3-3 LOF 50 1 1 0.35 0.8252032520325203 0.8916876574307305 0.8634146341463415 0.921875 0.23214435577392578
bpic13-0.3-3 LOF 50 1 1 0.4 0.8252032520325203 0.8916876574307305 0.8634146341463415 

bpic13-0.3-3 SVM 50 1 10 rbf auto 0.8252032520325203 0.8978622327790975 0.8253275109170306 0.984375 0.039290428161621094
bpic13-0.3-3 SVM 50 1 10 sigmoid scale 0.7398373983739838 0.8476190476190476 0.7807017543859649 0.9270833333333334 0.024959325790405273
bpic13-0.3-3 SVM 50 1 10 sigmoid auto 0.7479674796747967 0.8480392156862745 0.8009259259259259 0.9010416666666666 0.03575396537780762
bpic13-0.3-3 SVM 50 1 100 poly scale 0.8333333333333334 0.9026128266033255 0.8296943231441049 0.9895833333333334 0.03210783004760742
bpic13-0.3-3 SVM 50 1 100 poly auto 0.8089430894308943 0.8861985472154963 0.8280542986425339 0.953125 0.03091716766357422
bpic13-0.3-3 SVM 50 1 100 rbf scale 0.8373983739837398 0.9029126213592233 0.8454545454545455 0.96875 0.030916929244995117
bpic13-0.3-3 SVM 50 1 100 rbf auto 0.7886178861788617 0.8706467661691543 0.8333333333333334 0.9114583333333334 0.029920101165771484
bpic13-0.3-3 SVM 50 1 100 sigmoid scale 0.7479674796747967 0.849514563106796 0.7954545454545454 0.91

bpic13-0.3-3 OCSVM 50 1 0.4 rbf scale 0.5040650406504065 0.5793103448275861 0.8571428571428571 0.4375 0.015621662139892578
bpic13-0.3-3 OCSVM 50 1 0.4 rbf auto 0.4959349593495935 0.5724137931034483 0.8469387755102041 0.4322916666666667 0.013598442077636719
bpic13-0.3-3 OCSVM 50 1 0.4 sigmoid scale 0.6626016260162602 0.7821522309711286 0.7883597883597884 0.7760416666666666 0.013962745666503906
bpic13-0.3-3 OCSVM 50 1 0.4 sigmoid auto 0.5365853658536586 0.6369426751592356 0.819672131147541 0.5208333333333334 0.013962268829345703
bpic13-0.3-3 LOF 50 3 1 0.01 0.8048780487804879 0.8873239436619719 0.8076923076923077 0.984375 0.2433764934539795
bpic13-0.3-3 LOF 50 3 1 0.05 0.8333333333333334 0.8992628992628993 0.8511627906976744 0.953125 0.2298893928527832
bpic13-0.3-3 LOF 50 3 1 0.1 0.8333333333333334 0.8967254408060454 0.8682926829268293 0.9270833333333334 0.22402572631835938
bpic13-0.3-3 LOF 50 3 1 0.15 0.8333333333333334 0.8967254408060454 0.8682926829268293 0.9270833333333334 0.22647070

bpic13-0.3-3 SVM 50 3 0.1 sigmoid scale 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.03291201591491699
bpic13-0.3-3 SVM 50 3 0.1 sigmoid auto 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.02695608139038086
bpic13-0.3-3 SVM 50 3 1 poly scale 0.8252032520325203 0.8988235294117647 0.8197424892703863 0.9947916666666666 0.033835649490356445
bpic13-0.3-3 SVM 50 3 1 poly auto 0.8333333333333334 0.9035294117647059 0.8240343347639485 1.0 0.02890801429748535
bpic13-0.3-3 SVM 50 3 1 rbf scale 0.8252032520325203 0.8992974238875878 0.8170212765957446 1.0 0.029920101165771484
bpic13-0.3-3 SVM 50 3 1 rbf auto 0.8252032520325203 0.8992974238875878 0.8170212765957446 1.0 0.028922557830810547
bpic13-0.3-3 SVM 50 3 1 sigmoid scale 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.03166556358337402
bpic13-0.3-3 SVM 50 3 1 sigmoid auto 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.028922080993652344
bpic13-0.3-3 SVM 50 3 10 poly scale 0.81300

bpic13-0.3-3 OCSVM 50 3 0.2 rbf scale 0.540650406504065 0.6245847176079734 0.8623853211009175 0.4895833333333333 0.007978677749633789
bpic13-0.3-3 OCSVM 50 3 0.2 rbf auto 0.6951219512195121 0.7875354107648725 0.8633540372670807 0.7239583333333334 0.00797891616821289
bpic13-0.3-3 OCSVM 50 3 0.2 sigmoid scale 0.524390243902439 0.6666666666666665 0.7358490566037735 0.609375 0.007978677749633789
bpic13-0.3-3 OCSVM 50 3 0.2 sigmoid auto 0.7520325203252033 0.8398950131233595 0.8465608465608465 0.8333333333333334 0.008975982666015625
bpic13-0.3-3 OCSVM 50 3 0.25 poly scale 0.4796747967479675 0.564625850340136 0.8137254901960784 0.4322916666666667 0.008975744247436523
bpic13-0.3-3 OCSVM 50 3 0.25 poly auto 0.4796747967479675 0.564625850340136 0.8137254901960784 0.4322916666666667 0.008976221084594727
bpic13-0.3-3 OCSVM 50 3 0.25 rbf scale 0.5772357723577236 0.675 0.84375 0.5625 0.008975744247436523
bpic13-0.3-3 OCSVM 50 3 0.25 rbf auto 0.4349593495934959 0.4945454545454545 0.8192771084337349 0

...

bpic13-0.3-3 OCSVM 1000 3 0.35 rbf scale 0.6829268292682927 0.7732558139534884 0.875 0.6927083333333334 0.13837814331054688
bpic13-0.3-3 OCSVM 1000 3 0.35 rbf auto 0.6544715447154471 0.7477744807121661 0.8689655172413793 0.65625 0.15360403060913086
bpic13-0.3-3 OCSVM 1000 3 0.35 sigmoid scale 0.6829268292682927 0.8020304568527918 0.7821782178217822 0.8229166666666666 0.1713554859161377
bpic13-0.3-3 OCSVM 1000 3 0.35 sigmoid auto 0.6869918699186992 0.8040712468193383 0.7860696517412935 0.8229166666666666 0.15135502815246582
bpic13-0.3-3 OCSVM 1000 3 0.4 poly scale 0.4715447154471545 0.5357142857142857 0.8522727272727273 0.390625 0.2100358009338379
bpic13-0.3-3 OCSVM 1000 3 0.4 poly auto 0.45528455284552843 0.5144927536231885 0.8452380952380952 0.3697916666666667 0.21118879318237305
bpic13-0.3-3 OCSVM 1000 3 0.4 rbf scale 0.4186991869918699 0.4521072796934866 0.855072463768116 0.3072916666666667 0.18274259567260742
bpic13-0.3-3 OCSVM 1000 3 0.4 rbf auto 0.4105691056910569 0.4528301886792

bpic13-0.3-3 LOF 1000 5 250 0.35 0.7073170731707317 0.8085106382978724 0.8260869565217391 0.7916666666666666 0.2409203052520752
bpic13-0.3-3 LOF 1000 5 250 0.4 0.5853658536585366 0.7052023121387284 0.7922077922077922 0.6354166666666666 0.24051189422607422
bpic13-0.3-3 LOF 1000 5 250 auto 0.8130081300813008 0.8894230769230769 0.8258928571428571 0.9635416666666666 0.2608482837677002
bpic13-0.3-3 SVM 1000 5 0.1 poly scale 0.8252032520325203 0.8988235294117647 0.8197424892703863 0.9947916666666666 0.34459638595581055
bpic13-0.3-3 SVM 1000 5 0.1 poly auto 0.8211382113821138 0.897196261682243 0.8135593220338984 1.0 0.28464794158935547
bpic13-0.3-3 SVM 1000 5 0.1 rbf scale 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.283297061920166
bpic13-0.3-3 SVM 1000 5 0.1 rbf auto 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.2642326354980469
bpic13-0.3-3 SVM 1000 5 0.1 sigmoid scale 0.7804878048780488 0.8767123287671234 0.7804878048780488 1.0 0.3077116012573242
bpic13-

bpic13-0.3-3 OCSVM 1000 5 0.15 poly auto 0.45934959349593496 0.5521885521885521 0.780952380952381 0.4270833333333333 0.10454535484313965
bpic13-0.3-3 OCSVM 1000 5 0.15 rbf scale 0.6422764227642277 0.7471264367816092 0.8333333333333334 0.6770833333333334 0.08011150360107422
bpic13-0.3-3 OCSVM 1000 5 0.15 rbf auto 0.6260162601626016 0.7371428571428572 0.8164556962025317 0.671875 0.08466601371765137
bpic13-0.3-3 OCSVM 1000 5 0.15 sigmoid scale 0.5691056910569106 0.707182320441989 0.7529411764705882 0.6666666666666666 0.06664919853210449
bpic13-0.3-3 OCSVM 1000 5 0.15 sigmoid auto 0.7276422764227642 0.837772397094431 0.7828054298642534 0.9010416666666666 0.07901787757873535
bpic13-0.3-3 OCSVM 1000 5 0.2 poly scale 0.4715447154471545 0.5608108108108109 0.7980769230769231 0.4322916666666667 0.11683082580566406
bpic13-0.3-3 OCSVM 1000 5 0.2 poly auto 0.44715447154471544 0.5277777777777777 0.7916666666666666 0.3958333333333333 0.1002347469329834
bpic13-0.3-3 OCSVM 1000 5 0.2 rbf scale 0.662601

bpic13-0.3-3 LOF 1000 10 50 0.01 0.7845528455284553 0.8787185354691075 0.7836734693877551 1.0 0.26859450340270996
bpic13-0.3-3 LOF 1000 10 50 0.05 0.7682926829268293 0.8683602771362586 0.7800829875518672 0.9791666666666666 0.27106618881225586
bpic13-0.3-3 LOF 1000 10 50 0.1 0.7398373983739838 0.8431372549019608 0.7962962962962963 0.8958333333333334 0.26027989387512207
bpic13-0.3-3 LOF 1000 10 50 0.15 0.7520325203252033 0.8407310704960834 0.8429319371727748 0.8385416666666666 0.26496124267578125
bpic13-0.3-3 LOF 1000 10 50 0.2 0.7520325203252033 0.8407310704960834 0.8429319371727748 0.8385416666666666 0.2889587879180908
bpic13-0.3-3 LOF 1000 10 50 0.25 0.7235772357723578 0.8181818181818181 0.8406593406593407 0.796875 0.2906031608581543
bpic13-0.3-3 LOF 1000 10 50 0.3 0.7032520325203252 0.7988980716253443 0.847953216374269 0.7552083333333334 0.2563316822052002
bpic13-0.3-3 LOF 1000 10 50 0.35 0.6910569105691057 0.7865168539325843 0.8536585365853658 0.7291666666666666 0.26468944549560547


bpic13-0.3-3 SVM 1000 10 10000 sigmoid scale 0.7723577235772358 0.8697674418604651 0.7857142857142857 0.9739583333333334 0.29027652740478516
bpic13-0.3-3 SVM 1000 10 10000 sigmoid auto 0.7520325203252033 0.8557919621749408 0.7835497835497836 0.9427083333333334 0.3096354007720947
bpic13-0.3-3 SVM 1000 10 100000 poly scale 0.8252032520325203 0.8933002481389579 0.8530805687203792 0.9375 0.3169279098510742
bpic13-0.3-3 SVM 1000 10 100000 poly auto 0.7967479674796748 0.8737373737373737 0.8480392156862745 0.9010416666666666 0.31442761421203613
bpic13-0.3-3 SVM 1000 10 100000 rbf scale 0.7886178861788617 0.8737864077669902 0.8181818181818182 0.9375 0.29997682571411133
bpic13-0.3-3 SVM 1000 10 100000 rbf auto 0.7886178861788617 0.8725490196078431 0.8240740740740741 0.9270833333333334 0.29776477813720703
bpic13-0.3-3 SVM 1000 10 100000 sigmoid scale 0.7682926829268293 0.8671328671328671 0.7848101265822784 0.96875 0.3114349842071533
bpic13-0.3-3 SVM 1000 10 100000 sigmoid auto 0.7642276422764228

# anomaly detection and save metrics

In [78]:
# LOF_save_dir = './detect_metrics_results_all/LOF/'
# SVM_save_dir = './detect_metrics_results_all/SVM/'
# OCSVM_save_dir = './detect_metrics_results_all/OCSVM/'
LOF_save_dir = './detect_metrics_results_all/LOF_BPIC13OpenProblems/'
SVM_save_dir = './detect_metrics_results_all/SVM_BPIC13OpenProblems/'
OCSVM_save_dir = './detect_metrics_results_all/OCSVM_BPIC13OpenProblems/'

scl = StandardScaler()
# path = 'preprocessed_csv'
path = 'preprocessed_csv_BPIC13OpenProblems'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    print(log, log_name)
    dataset_name = log_name.split('-')[0]
    anomaly_ratio = log_name.split('-')[1]
    
    df = read_log(path, log)

    # process cases and labels
    cases, y = cases_y_list(df)
    del df

#     在BPM应用程序中，建议使用更小的向量50和窗口大小
#     for size_param in [50, 100, 150, 200, 250, 300, 400, 500, 750, 1000]: # word2vec的向量大小
#         for window_param in [1, 3, 5, 10]: # word2vec的窗口大小，论文实验表明窗口大小不影响性能。

    # 使用论文中通过实验获取的最佳向量大小和窗口大小
    size_param = 50
    window_param = 1
    
    # generate model
    model = create_models(cases, size_param, window_param, 1)

    # calculating the average feature vector for each sentence (trace)
    vectors = average_feature_vector(cases, model)
    # normalization
    vectors = scl.fit_transform(vectors)

    X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1)
    X_train = np.array(X_train)
    positive_mask = np.array(y_train) == 1
    
    LOF_neighbors_param = 1
    LOF_contamination_param = 0.01
    lof = LocalOutlierFactor(n_neighbors=LOF_neighbors_param, contamination=LOF_contamination_param, novelty=True, n_jobs=n_workers)
    lof.fit(X_train[positive_mask])
    y_pred = lof.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    LOF_detect_metrics_df = pd.DataFrame({
        'dataset_name':[dataset_name],
        'anomaly_ratio':[anomaly_ratio],
        'algorithm': ['LOF'],
        'F1-score':[fscore],
        'precision':[precision],
        'recall':[recall],
        'accuracy':[acc]
    })
    LOF_detect_metrics_df.to_csv(LOF_save_dir+log_name+'.csv')
    print(log_name, 'LOF', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('---------------------------------------------')
    
    SVM_c_param = 1000
    SVM_kernel_param = 'poly'
    SVM_gamma_param = 'scale'
    n_estimators = 10
    svm = BaggingClassifier(base_estimator=SVC(kernel=SVM_kernel_param, C=SVM_c_param, gamma=SVM_gamma_param, max_iter=1000000), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=n_workers)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    SVM_detect_metrics_df = pd.DataFrame({
        'dataset_name':[dataset_name],
        'anomaly_ratio':[anomaly_ratio],
        'algorithm': ['SVM'],
        'F1-score':[fscore],
        'precision':[precision],
        'recall':[recall],
        'accuracy':[acc]
    })
    SVM_detect_metrics_df.to_csv(SVM_save_dir+log_name+'.csv')
    print(log_name, 'SVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('---------------------------------------------')
             
    OCSVM_nu_param = 0.01
    # 论文中kernel_param，gamma_param两个参数没有说明具体哪个值更好，因此本实验中选择默认值,如下所示
    OCSVM_kernel_param = 'rbf'
    OCSVM_gamma_param = 'scale'
    ocsvm = OneClassSVM(nu=OCSVM_nu_param, kernel=OCSVM_kernel_param, gamma=OCSVM_gamma_param, max_iter=1000000)
    ocsvm.fit(X_train[positive_mask])
    y_pred = ocsvm.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    OCSVM_detect_metrics_df = pd.DataFrame({
        'dataset_name':[dataset_name],
        'anomaly_ratio':[anomaly_ratio],
        'algorithm': ['OCSVM'],
        'F1-score':[fscore],
        'precision':[precision],
        'recall':[recall],
        'accuracy':[acc]
    })
    OCSVM_detect_metrics_df.to_csv(OCSVM_save_dir+log_name+'.csv')
    print(log_name, 'OCSVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)
    print('----------------------------------------------------------------')
    print('***********************************Finished log: ', log_name)


BPIC13OpenProblems-0.05.csv BPIC13OpenProblems-0.05
BPIC13OpenProblems-0.05 LOF  acc= 0.9634146341463414  fscore= 0.9813664596273292  precision= 0.9793388429752066  recall= 0.983402489626556
---------------------------------------------
BPIC13OpenProblems-0.05 SVM  acc= 0.975609756097561  fscore= 0.9876543209876544  precision= 0.9795918367346939  recall= 0.995850622406639
---------------------------------------------
BPIC13OpenProblems-0.05 OCSVM  acc= 0.8821138211382114  fscore= 0.936542669584245  precision= 0.9907407407407407  recall= 0.8879668049792531
----------------------------------------------------------------
***********************************Finished log:  BPIC13OpenProblems-0.05
BPIC13OpenProblems-0.1.csv BPIC13OpenProblems-0.1
BPIC13OpenProblems-0.1 LOF  acc= 0.9390243902439024  fscore= 0.9680170575692963  precision= 0.9497907949790795  recall= 0.9869565217391304
---------------------------------------------
BPIC13OpenProblems-0.1 SVM  acc= 0.9512195121951219  fscore= 0.9

# summary all detect metrics

In [ ]:
OCSVM_detect_metrics_df = pd.DataFrame({
        'dataset_name':[dataset_name],
        'anomaly_ratio':[anomaly_ratio],
        'algorithm': ['OCSVM'],
        'F1-score':[fscore],
        'precision':[precision],
        'recall':[recall],
        'accuracy':[acc]
    })

In [89]:
LOF_save_dir = './detect_metrics_results_all/LOF/'
SVM_save_dir = './detect_metrics_results_all/SVM/'
OCSVM_save_dir = './detect_metrics_results_all/OCSVM/'

# all_detect_metrics_df = pd.DataFrame({'dataset_name','anomaly_ratio','algorithm','F1-score','precision','recall','accuracy'})
all_detect_metrics_df = pd.DataFrame(columns=['dataset_name', 'anomaly_ratio', 'algorithm', 
                                              'F1-score', 'precision', 'recall', 'accuracy']) 
save_list = [LOF_save_dir, SVM_save_dir, OCSVM_save_dir]
# save_list = [LOF_save_dir]
for i in range(len(save_list)):
    for csv_name in os.listdir(save_list[i]):
        result_df = pd.read_csv(save_list[i] + csv_name, index_col=0)
        all_detect_metrics_df = all_detect_metrics_df.append(result_df)
#     print('-------------------------')

all_detect_metrics_df.to_csv('./detect_metrics_results_all/summary/detect_metrics_result_summary.csv', index=False)

In [90]:
all_detect_metrics_df

,dataset_name,anomaly_ratio,algorithm,F1-score,precision,recall,accuracy
0,BPIC13ClosedProblems,0.05,LOF,0.971165,0.954649,0.988263,0.944072
0,BPIC13ClosedProblems,0.10,LOF,0.958084,0.932401,0.985222,0.921700
0,BPIC13ClosedProblems,0.20,LOF,0.913366,0.850230,0.986631,0.843400
0,BPIC13ClosedProblems,0.30,LOF,0.885906,0.802920,0.988024,0.809843
0,BPIC13Incidents,0.05,LOF,0.976077,0.960108,0.992586,0.953683
...,...,...,...,...,...,...,...
0,BPIC20RequestForPayment,0.30,OCSVM,0.719553,0.784091,0.664831,0.635528
0,Helpdesk,0.05,OCSVM,0.545949,0.941839,0.384380,0.392285
0,Helpdesk,0.10,OCSVM,0.956832,0.948195,0.965627,0.920670
0,Helpdesk,0.20,OCSVM,0.894900,0.869828,0.921461,0.827511


# test

In [71]:
scl = StandardScaler()
# path = 'data_test'
# path = 'second_test_error'
path = 'preprocessed_csv_BPIC13OpenProblems'
for log in os.listdir(path):
    log_name = log.split('.csv')[0]
    start_time_log = time.time()
    print('log_name=', log_name)
    # reads event log
    df = read_log(path, log)

    # process cases and labels
    cases, y = cases_y_list(df)
    del df

    size_param = 50
    window_param = 1
    model = create_models(cases, size_param, window_param, 1)

    # calculating the average feature vector for each sentence (trace)
    vectors = average_feature_vector(cases, model)
    # normalization
    vectors = scl.fit_transform(vectors)

    X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=1)
    X_train = np.array(X_train)
    positive_mask = np.array(y_train) == 1
    
    
    SVM_c_param = 1000
    SVM_kernel_param = 'poly'
    SVM_gamma_param = 'scale'
    n_estimators = 10
    start_time = time.time()
    svm = BaggingClassifier(base_estimator=SVC(kernel=SVM_kernel_param, C=SVM_c_param, 
                                               gamma=SVM_gamma_param, max_iter=1000000), 
                            max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=n_workers)
    print(type(X_train), X_train.shape, type(y_train), len(y_train))
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    acc, fscore, precision, recall = compute_metrics(y_test, y_pred)
    elapsed_time = time.time() - start_time
    print(log_name, 'SVM', ' acc=', acc, ' fscore=', fscore, ' precision=', precision, ' recall=', recall)



log_name= BPIC13OpenProblems-0.1
<class 'numpy.ndarray'> (573, 50) <class 'list'> 573
BPIC13OpenProblems-0.1 SVM  acc= 0.9471544715447154  fscore= 0.9725158562367865  precision= 0.9465020576131687  recall= 1.0
log_name= BPIC13OpenProblems-0.3
<class 'numpy.ndarray'> (573, 50) <class 'list'> 573
BPIC13OpenProblems-0.3 SVM  acc= 0.8333333333333334  fscore= 0.9044289044289044  precision= 0.8622222222222222  recall= 0.9509803921568627
